Clear Previous Model and Cache


**NOTE** Use Only if required

In [ ]:
import gc
import torch

# Clear any existing models and empty the cache
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install transformers accelerate bitsandbytes datasets peft sacrebleu bert-score codebleu

Importing libraries and loading drive

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from sacrebleu import corpus_bleu, corpus_chrf
from bert_score import score as bert_score
from codebleu import codebleu
import torch
import numpy as np

In [ ]:
from google.colab import drive
# Mount Google Drive if needed
drive.mount('/content/drive')

Mounted at /content/drive


Loading Dataset

In [ ]:
import json
from datasets import Dataset
# Load the JSON data
data_path = '' # Path to the JSON file
with open(data_path, 'r') as f:
    data = json.load(f)

# Convert to Hugging Face Dataset format
hf_dataset = Dataset.from_list(data)


In [ ]:
# Load dataset and split into training, validation, and test sets (80-10-10 split)
dataset = hf_dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
temp_eval = dataset["test"].train_test_split(test_size=0.5)
val_dataset = temp_eval["train"]
test_dataset = temp_eval["test"]

In [ ]:
'''
import random

# Ensure test_dataset is in list format for sampling
test_dataset = list(test_dataset)

# Select 5 random samples from the test dataset and store them in subset_test_dataset
subset_test_dataset = random.sample(test_dataset, 6)

# Display the selected samples (optional)
for i, sample in enumerate(subset_test_dataset, start=1):
    print(f"Sample {i}:")
    print(f"Prompt: {sample['prompt']}")
    print(f"Reference Code: {sample['completion']}")
    print("-" * 50)
'''

Sample 1:
Prompt: Create a header for a portfolio website with a focus on personal branding using a minimalist and modern look. Integrate HTML components like a headline, description, and call-to-action buttons, along with social media links. Use CSS for cohesive typography, vibrant color highlights, and a user-centered layout. Aim for visual hierarchy, intuitive navigation, and a responsive design to enhance user engagement.


Reference Code: <html>
<head>
<style>
.dimas-ardiansyah {
  	color: #7456ff;
}
.i-am-dimas-container {
  	position: absolute;
  	top: 64px;
  	left: 0px;
  	font-weight: 600;
}
.ui {
  	font-weight: 500;
}
.span {
  	font-weight: 500;
  	font-family: 'Noto Sans';
}
.ui-ux-container {
  	position: absolute;
  	top: 152px;
  	left: 1px;
  	font-size: 20px;
  	color: #999;
}
.hello-buds {
  	position: absolute;
  	top: 0px;
  	left: 0px;
  	font-size: 36px;
  	font-weight: 600;
}
.header {
  	position: absolute;
  	top: 0px;
  	left: 0px;
  	width: 619px;
  	height

In [ ]:
print(subset_test_dataset[1])

{'prompt': 'Create a header of a Professional Portfolio Website focusing on a creative and vibrant layout. Utilize HTML for structured sections like About Me, Services, and buttons such as Hire Me and Download CV. Style with CSS to emphasize a modern UI, incorporating color gradients, rounded buttons, and clear typography, enhancing readability and user interaction while ensuring visual consistency.\n\n', 'completion': '<html>\n<head>\n<style>\n.frame-work-1 {\n  \tposition: absolute;\n  \ttop: 156px;\n  \tleft: 757px;\n  \twidth: 617px;\n  \theight: 576.4px;\n  \tobject-fit: cover;\n}\n.group-child {\n  \tposition: absolute;\n  \ttop: 0px;\n  \tleft: 0px;\n  \tborder-radius: 50%;\n  \tbackground-color: #f9c61b;\n  \tborder: 4px solid #fffdfd;\n  \tbox-sizing: border-box;\n  \twidth: 57px;\n  \theight: 57px;\n}\n.group-item {\n  \tposition: absolute;\n  \ttop: 0px;\n  \tleft: 38.98px;\n  \tborder-radius: 50%;\n  \tbackground-color: #1188e7;\n  \tborder: 4px solid #f8f8f8;\n  \tbox-sizi

In [ ]:
import json

# Define the path where the JSON file is saved
load_path = "/content/drive/MyDrive/StarryStudioAI/subset_test_dataset.json"

# Load the JSON file into the subset_test_dataset variable
with open(load_path, 'r') as f:
    subset_test_dataset = json.load(f)

print("subset_test_dataset has been loaded successfully.")


subset_test_dataset has been loaded successfully.


HuggingFace login

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

Initialize Tokenizer and Set Padding Token

In [ ]:
# Initialize the tokenizer
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set a padding token if it does not already exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

    # Tokenization function
def preprocess_function(examples):
    inputs = examples['prompt']
    outputs = examples['completion']
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=1024)
    labels = tokenizer(outputs, padding="max_length", truncation=True, max_length=1024).input_ids
    model_inputs["labels"] = labels
    return model_inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Data Preprocessing & Tokenize Dataset

In [ ]:
# Apply preprocessing to the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = val_dataset.map(preprocess_function, batched=True)
# If you have a test dataset
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

In [ ]:
'''
# Tokenization function
def preprocess_function(examples):
    inputs = examples['prompt']
    outputs = examples['completion']

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=1024,
        return_tensors="pt"
    )

    # Tokenize outputs (labels) and set padding tokens to -100 for loss calculation
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            outputs,
            padding="max_length",
            truncation=True,
            max_length=1024,
            return_tensors="pt"
        ).input_ids

    # Replace padding token IDs in labels with -100
    labels[labels == tokenizer.pad_token_id] = -100

    model_inputs["labels"] = labels
    return model_inputs

# Apply preprocessing to the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = val_dataset.map(preprocess_function, batched=True)
# If you have a test dataset
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
'''

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Load Model and Configure LoRA with PEFT

In [ ]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

# Load the model without specifying device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Specify half-precision for memory efficiency
)

# Move the model to CUDA explicitly
model.to('cuda')

# Set up LoRA configuration for PEFT
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

# Apply LoRA to the model using PEFT
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
!pip install cssutils

In [ ]:
!pip install wandb


In [ ]:
import wandb
import os
!wandb login --relogin


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from transformers import TrainingArguments
# Configure training arguments
training_args = TrainingArguments(
    output_dir="./results",               # Directory for saving results
    per_device_train_batch_size=1,        # Batch size per device (adjust if needed)
    gradient_accumulation_steps=8,        # Effective batch size of 8
    learning_rate=5e-5,                   # Learning rate
    num_train_epochs=4,                   # Number of training epochs
    fp16=True,                            # Mixed precision for efficient GPU usage
    save_steps=500,                       # Save checkpoint every 500 steps
    logging_steps=100,                    # Log training progress every 100 steps
    evaluation_strategy="steps",          # Evaluate during training
    save_total_limit=2,                   # Keep only the 2 most recent checkpoints
    load_best_model_at_end=True,
    report_to="wandb" # Load the best model at the end
)

from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Initialize Trainer and Start Training

In [ ]:
# Start the fine-tuning process
trainer.train()

In [ ]:
# After training, save the model
trainer.save_model("/StarryStudioAI/M_fine_tuned_model")
tokenizer.save_pretrained("StarryStudioAI/MT_fine_tuned_model")

('/content/drive/MyDrive/StarryStudioAI/MT_fine_tuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/StarryStudioAI/MT_fine_tuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/StarryStudioAI/MT_fine_tuned_model/tokenizer.json')

In [ ]:
import gc
import torch

# Clear any existing models and empty the cache
gc.collect()
torch.cuda.empty_cache()

In [ ]:
'''
# Evaluate on the training set after training if needed
train_results = trainer.evaluate(eval_dataset=tokenized_train_dataset)
print("Training Results:", train_results)
'''

In [ ]:
'''
# Evaluate on the test set after training
test_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print("Test Results:", test_results)
'''

Test the Model with an Inference Example

In [ ]:
# Test with a sample prompt to verify HTML/CSS generation
test_prompt = "Generate a modern, minimalistic HTML header layout for the design brand StarryStudioAI. The header should include the brand logo svg and navigation links: Home, About, and Contact. Use clean, responsive design principles and simple CSS styling."
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=1024)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated HTML/CSS:\n", generated_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated HTML/CSS:
 Generate a modern, minimalistic HTML header layout for the design brand StarryStudioAI. The header should include the brand logo svg and navigation links: Home, About, and Contact. Use clean, responsive design principles and simple CSS styling. 

### Header Layout
```html
<header class="header">
  <div class="logo">
    <svg class="logo-svg" width="50" height="50" viewBox="0 0 100 100">
      <circle cx="50" cy="50" r="45" stroke-width="10" stroke="currentColor" stroke-miterlimit="10" fill="none" />
      <circle cx="50" cy="50" r="45" stroke-width="10" stroke="currentColor" stroke-miterlimit="10" fill="none" />
      <circle cx="50" cy="50" r="45" stroke-width="10" stroke="currentColor" stroke-miterlimit="10" fill="none" />
    </svg>
  </div>
  <nav class="nav">
    <ul>
      <li><a href="index.html">Home</a></li>
      <li><a href="about.html">About</a></li>
      <li><a href="contact.html">Contact</a></li>
    </ul>
  </nav>
</header>
```

### CSS
```css
heade

In [ ]:
# Further refined prompt for single, concise HTML/CSS code output
test_prompt = (
    "Generate only the HTML and CSS code for a modern, minimalistic header layout for the design brand StarryStudioAI. The header should include a brand logo and navigation links for Home, About, and Contact. Use responsive design principles with clean, simple CSS styling. Do not include any explanations, extra text, or repeated sections—only provide the HTML and CSS code."
)

# Generate the code
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=1024)  # Adjust max_new_tokens as needed

# Decode and display the generated HTML/CSS code
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated HTML/CSS Code:\n", generated_text)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated HTML/CSS Code:
 Generate only the HTML and CSS code for a modern, minimalistic header layout for the design brand StarryStudioAI. The header should include a brand logo and navigation links for Home, About, and Contact. Use responsive design principles with clean, simple CSS styling. Do not include any explanations, extra text, or repeated sections—only provide the HTML and CSS code.html
```html
<header>
  <div class="header__logo">
    <a href="index.html" class="logo">
      <img src="logo.png" alt="Logo">
    </a>
  </div>
  <div class="header__nav">
    <nav>
      <ul>
        <li><a href="index.html">Home</a></li>
        <li><a href="about.html">About</a></li>
        <li><a href="contact.html">Contact</a></li>
      </ul>
    </nav>
  </div>
</header>
```
.css
```css
header {
  display: flex;
  justify-content: center;
  align-items: center;
  padding: 1em;
  background: #f2f2f2;
  border-bottom: 1px solid #ccc;
}

.header__logo {
  display: flex;
  align-items: cente

### RUBY (AST+STS)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from bs4 import BeautifulSoup
from difflib import SequenceMatcher
import tinycss2
import torch
import nltk
from nltk.tokenize import word_tokenize

# Ensure NLTK tokenizer is available
nltk.download('punkt')

# Load the tokenizer and model
tokenizer_path = "StarryStudioAI/MT_fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model_path = "StarryStudioAI/M_fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to('cpu')  # Load to CPU for evaluation if GPU is limited

# STS-based similarity calculation using NLTK tokenizer
def calculate_sts_similarity(generated_code, reference_code):
    generated_tokens = word_tokenize(generated_code)  # Use NLTK word_tokenize
    reference_tokens = word_tokenize(reference_code)
    matcher = SequenceMatcher(None, generated_tokens, reference_tokens)
    return matcher.ratio()

# AST-based similarity for HTML using BeautifulSoup
def calculate_ast_similarity_html(generated_code, reference_code):
    generated_tree = BeautifulSoup(generated_code, "html.parser")
    reference_tree = BeautifulSoup(reference_code, "html.parser")

    def compare_trees(node1, node2):
        if node1.name != node2.name:
            return 0  # Different tags
        score = 1  # Initial similarity score for matching tag names
        if node1.attrs == node2.attrs:
            score += 1  # Matching attributes increase score
        for child1, child2 in zip(node1.children, node2.children):
            score += compare_trees(child1, child2)  # Recursive comparison
        return score

    return compare_trees(generated_tree, reference_tree) / max(len(generated_tree), len(reference_tree))

# AST-based similarity for CSS using TinyCSS2
def calculate_ast_similarity_css(generated_code, reference_code):
    generated_rules = tinycss2.parse_stylesheet(generated_code)
    reference_rules = tinycss2.parse_stylesheet(reference_code)

    def compare_css_rules(rule1, rule2):
        if rule1.type != rule2.type:
            return 0
        score = 1  # Matching rule type
        if rule1.content == rule2.content:
            score += 1  # Matching content
        return score

    score = 0
    for rule1, rule2 in zip(generated_rules, reference_rules):
        score += compare_css_rules(rule1, rule2)
    return score / max(len(generated_rules), len(reference_rules))

# Wrapper for RUBY metric with AST and STS options, assuming both HTML and CSS
def ruby_metric(generated_code, reference_code):
    try:
        # Calculate AST similarity for both HTML and CSS
        ast_html_score = calculate_ast_similarity_html(generated_code, reference_code)
        ast_css_score = calculate_ast_similarity_css(generated_code, reference_code)

        # Average HTML and CSS AST scores
        ast_score = (ast_html_score + ast_css_score) / 2
    except Exception as e:
        print(f"AST comparison failed: {e}")
        ast_score = 0  # Fallback to 0 if AST fails

    # Calculate STS score
    sts_score = calculate_sts_similarity(generated_code, reference_code)

    # Return AST and STS scores along with the combined average score
    combined_score = (ast_score + sts_score) / 2
    return ast_score, sts_score, combined_score

def evaluate_ruby_subset(subset_test_dataset, num_samples=1):
    # Limit to num_samples or the length of the dataset, whichever is smaller
    sample_dataset = subset_test_dataset[:num_samples]

    ast_scores = []
    sts_scores = []
    combined_scores = []

    for example in sample_dataset:
        prompt = example['prompt']
        reference_code = example['completion']

        # Tokenize and generate output
        inputs = tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=1024)
        generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

        # Calculate RUBY metric scores
        ast_score, sts_score, combined_score = ruby_metric(generated_code, reference_code)
        ast_scores.append(ast_score)
        sts_scores.append(sts_score)
        combined_scores.append(combined_score)

        # Print each result
        print(f"Prompt: {prompt}")
        print(f"Generated Code: {generated_code}")
        print(f"Reference Code: {reference_code}")
        print(f"AST Score (Average HTML & CSS): {ast_score}")
        print(f"STS Score: {sts_score}")
        print(f"Combined Score: {combined_score}")
        print("-" * 50)

    # Calculate and print average scores
    avg_ast_score = sum(ast_scores) / len(ast_scores) if ast_scores else 0
    avg_sts_score = sum(sts_scores) / len(sts_scores) if sts_scores else 0
    avg_combined_score = sum(combined_scores) / len(combined_scores) if combined_scores else 0

    print(f"Average AST Score: {avg_ast_score}")
    print(f"Average STS Score: {avg_sts_score}")
    print(f"Average Combined Score: {avg_combined_score}")

    return avg_ast_score, avg_sts_score, avg_combined_score
'''
# Test on only one example
print("Evaluating on one sample...")
avg_ast_score, avg_sts_score, avg_combined_score = evaluate_ruby_subset(subset_test_dataset, num_samples=1)
'''
# After confirming it works, evaluate on all examples in the subset
print("\nEvaluating on all samples...")
avg_ast_score, avg_sts_score, avg_combined_score = evaluate_ruby_subset(subset_test_dataset, num_samples=len(subset_test_dataset))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Evaluating on all samples...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

AST comparison failed: 'WhitespaceToken' object has no attribute 'content'
Prompt: Create a header of a fashion e-commerce website, featuring a clean, modern look. Include an announcement bar, logo, navigation menu for New Arrivals, Swimwear, Collections, About Us, and Demos, a search icon, account link, and cart icon. Use responsive design, bold typography, intuitive layout, and strategically placed call-to-action buttons to enhance user experience and engagement.


Generated Code: Create a header of a fashion e-commerce website, featuring a clean, modern look. Include an announcement bar, logo, navigation menu for New Arrivals, Swimwear, Collections, About Us, and Demos, a search icon, account link, and cart icon. Use responsive design, bold typography, intuitive layout, and strategically placed call-to-action buttons to enhance user experience and engagement.

**Header HTML**
```html
<header class="header">
  <div class="container">
    <div class="row">
      <div class="col-12">
 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


AST comparison failed: 'WhitespaceToken' object has no attribute 'content'
Prompt: Create a header of a Portfolio Website for a Freelance UI/UX Designer with a modern and visually dynamic look. Implement hero imagery, impactful typography, clear call to action buttons, and a well-structured introduction using HTML and CSS. Emphasize visual hierarchy, white space, and responsive design, adhering to UI/UX principles to enhance user engagement and accessibility.


Generated Code: Create a header of a Portfolio Website for a Freelance UI/UX Designer with a modern and visually dynamic look. Implement hero imagery, impactful typography, clear call to action buttons, and a well-structured introduction using HTML and CSS. Emphasize visual hierarchy, white space, and responsive design, adhering to UI/UX principles to enhance user engagement and accessibility.

### Header HTML
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=devi

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


AST comparison failed: 'WhitespaceToken' object has no attribute 'content'
Prompt: Create a header of a Creative Agency Website, emphasizing a modern and minimalist design. Utilize HTML and CSS to structure a responsive navigation bar with bold branding, clear call-to-actions, and intuitive menu links. Implement visually striking elements, balanced whitespace, and consistent typography to enhance user engagement while adhering to UI/UX best practices for accessibility and seamless interaction.


Generated Code: Create a header of a Creative Agency Website, emphasizing a modern and minimalist design. Utilize HTML and CSS to structure a responsive navigation bar with bold branding, clear call-to-actions, and intuitive menu links. Implement visually striking elements, balanced whitespace, and consistent typography to enhance user engagement while adhering to UI/UX best practices for accessibility and seamless interaction.

### HTML Header Structure
```html
<header class="header-container"

###BERTscore

In [ ]:
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from bert_score import score as bert_score
import torch

# Load the tokenizer and model
tokenizer_path = "StarryStudioAI/MT_fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model_path = "StarryStudioAI/M_fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to('cpu')  # Load to CPU for evaluation if GPU is limited

# Load the subset_test_dataset JSON file
with open('StarryStudioAI/starcoder_training_data_200.json', 'r') as f:
    subset_test_dataset = json.load(f)

# Function to evaluate BERTScore on a sample of the test dataset
def evaluate_bertscore_sample(subset_test_dataset, sample_size=6):
    sample_dataset = subset_test_dataset[:sample_size]

    # Initialize lists for storing scores
    precisions, recalls, f1_scores = [], [], []

    for example in sample_dataset:
        prompt = example['prompt']
        reference_code = example['completion']

        # Tokenize and generate output
        inputs = tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=1024)
        generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

        # Calculate BERTScore
        P, R, F1 = bert_score([generated_code], [reference_code], lang="en", verbose=True)
        precisions.append(P.mean().item())
        recalls.append(R.mean().item())
        f1_scores.append(F1.mean().item())

        # Print each result
        print(f"Prompt: {prompt}")
        print(f"Generated Code: {generated_code}")
        print(f"Reference Code: {reference_code}")
        print(f"BERTScore Precision: {P.mean().item()}")
        print(f"BERTScore Recall: {R.mean().item()}")
        print(f"BERTScore F1: {F1.mean().item()}")
        print("-" * 50)

    # Calculate and print average BERTScore for the sample
    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0
    avg_f1_score = sum(f1_scores) / len(f1_scores) if f1_scores else 0

    print(f"Average BERTScore Precision: {avg_precision}")
    print(f"Average BERTScore Recall: {avg_recall}")
    print(f"Average BERTScore F1: {avg_f1_score}")

    return avg_precision, avg_recall, avg_f1_score

# Run BERTScore evaluation on a sample of the test dataset
avg_precision, avg_recall, avg_f1_score = evaluate_bertscore_sample(subset_test_dataset, sample_size=6)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.29 seconds, 3.45 sentences/sec
Prompt: Create a header for a SaaS Product website in the Design Tools subcategory with a Clean and Contemporary style. Utilize bold typography, clear navigation menu, strategic use of CTAs, and consistent iconography. Implement with flexbox or grid, ensuring color contrast, effective whitespace usage, and attention to visual hierarchy while maintaining a fully responsive layout for optimal user experience across devices.


Generated Code: Create a header for a SaaS Product website in the Design Tools subcategory with a Clean and Contemporary style. Utilize bold typography, clear navigation menu, strategic use of CTAs, and consistent iconography. Implement with flexbox or grid, ensuring color contrast, effective whitespace usage, and attention to visual hierarchy while maintaining a fully responsive layout for optimal user experience across devices.

**Header Header Header Header Header Header Header Header Header Header Header Header Header Hea

###CHRF


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from sacrebleu import corpus_chrf
import torch

# Load the tokenizer and model
tokenizer_path = "StarryStudioAI/MT_fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model_path = "StarryStudioAI/M_fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to('cpu')  # Load to CPU for evaluation if GPU is limited
# Load the subset_test_dataset JSON file
with open('StarryStudioAI/starcoder_training_data_200.json', 'r') as f:
    subset_test_dataset = json.load(f)
# Function to evaluate ChrF on a specific subset of the test dataset
def evaluate_chrf_sample(subset_test_dataset, sample_size=6):
    # Use a fixed slice of the dataset rather than random selection
    sample_dataset = subset_test_dataset[:sample_size]

    chrf_scores = []

    for example in sample_dataset:
        prompt = example['prompt']
        reference_code = example['completion']

        # Tokenize and generate output
        inputs = tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=1024)
        generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

        # Calculate ChrF score
        chrf_score = corpus_chrf([generated_code], [[reference_code]]).score
        chrf_scores.append(chrf_score)

        # Print each result
        print(f"Prompt: {prompt}")
        print(f"Generated Code: {generated_code}")
        print(f"Reference Code: {reference_code}")
        print(f"ChrF Score: {chrf_score}")
        print("-" * 50)

    # Calculate and print the average ChrF score for the sample
    avg_chrf_score = sum(chrf_scores) / len(chrf_scores) if chrf_scores else 0
    print(f"Average ChrF Score: {avg_chrf_score}")

    return avg_chrf_score

# Run ChrF evaluation on a fixed subset of the test dataset
print("Evaluating ChrF on a fixed subset of the test dataset...")
avg_chrf_score = evaluate_chrf_sample(subset_test_dataset, sample_size=6)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Evaluating ChrF on a fixed subset of the test dataset...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header of a personal portfolio website with a modern, clean aesthetic. Utilize HTML for logo, navigation bar, and concise intro sections, each with distinct calls-to-action. Employ CSS to emphasize typography, color contrasts, and hover interactions, focusing on readability and intuitive layout for seamless navigation while aligning with UI/UX best practices.


Generated Code: Create a header of a personal portfolio website with a modern, clean aesthetic. Utilize HTML for logo, navigation bar, and concise intro sections, each with distinct calls-to-action. Employ CSS to emphasize typography, color contrasts, and hover interactions, focusing on readability and intuitive layout for seamless navigation while aligning with UI/UX best practices.

*   **Logo:** The website features a stylized logo of a compass. The logo has a solid blue color and the compass image is a transparent PNG. The logo has the same vertical orientation as the main header. 
*     **Navigation Bar:** 

...

Prompt: Create a header for a Portfolio website under the Creative Professional subcategory with a Bold and Modern design. Integrate hero text, a captivating call to action button, and clear typography hierarchy. Utilize flexbox for layout, maintain a consistent color palette, employ whitespace for clarity, and ensure responsive design with attention to alignment, padding, and interactive hover states to elevate user experience.


Generated Code: Create a header for a Portfolio website under the Creative Professional subcategory with a Bold and Modern design. Integrate hero text, a captivating call to action button, and clear typography hierarchy. Utilize flexbox for layout, maintain a consistent color palette, employ whitespace for clarity, and ensure responsive design with attention to alignment, padding, and interactive hover states to elevate user experience.

**Header:**
  - **Title:**
    - 100px-width font-size
    - 500px-width font-size
    - 100px-width font-size
    - 500px-

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header for a portfolio website in the "Freelance Designer" category, featuring a sleek, modern layout with a logo, navigation menu, and a call-to-action button using HTML and CSS; ensure a responsive flexbox design, consistent typography ('Poppins' and 'Work Sans'), color contrast, padding, and spacing for an optimal UI/UX.


Generated Code: Create a header for a portfolio website in the "Freelance Designer" category, featuring a sleek, modern layout with a logo, navigation menu, and a call-to-action button using HTML and CSS; ensure a responsive flexbox design, consistent typography ('Poppins' and 'Work Sans'), color contrast, padding, and spacing for an optimal UI/UX.

**Header Header
  1. 0.5
  2. 1
  3. 1.5
  4. 2
  5. 3
  6. 4
  7. 5
  8. 6
  9. 7
  10. 8
  11. 9
  12. 10
  13. 11
  14. 12
  15. 13
  16. 14
  17. 15
  18. 16
  19. 17
  20. 18
  21. 19
  22. 20
  23. 21
  24. 22
  25. 23
  26. 24
  27. 25
  28. 26
  29. 27
  30. 28
  31. 29
  32. 30
  33. 31
  34. 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header of a Personal Portfolio Website with a modern and minimalistic look. Utilize HTML and CSS to feature a hero section, navigation menu, profile image, service highlights, and social media integration. Use call to action buttons like Download CV and Contact Me to enhance UX. Ensure all components align with UI/UX principles for clarity and engagement.


Generated Code: Create a header of a Personal Portfolio Website with a modern and minimalistic look. Utilize HTML and CSS to feature a hero section, navigation menu, profile image, service highlights, and social media integration. Use call to action buttons like Download CV and Contact Me to enhance UX. Ensure all components align with UI/UX principles for clarity and engagement.

**Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header Header

###BLUE


In [ ]:
import gc
import torch

# Clear any existing models and empty the cache
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from sacrebleu import corpus_bleu
import torch

# Load the tokenizer and model
tokenizer_path = "StarryStudioAI/MT_fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model_path = "StarryStudioAI/M_fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(model_path)

model.to('cpu')  # Ensure model is on CPU

# Load the subset_test_dataset JSON file
with open('StarryStudioAI/starcoder_training_data_200.json', 'r') as f:
    subset_test_dataset = json.load(f)

# Function to evaluate BLEU on a sample of the test dataset
def evaluate_bleu_sample(subset_test_dataset, sample_size=6):
    sample_dataset = subset_test_dataset[:sample_size]

    predictions = []
    references = []

    for example in sample_dataset:
        prompt = example['prompt']
        reference_code = example['completion']

        # Tokenize and generate output
        inputs = tokenizer(prompt, return_tensors="pt").to('cpu')  # Ensure inputs are on CPU
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=1024)
        generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

        # Collect predictions and references for BLEU calculation
        predictions.append(generated_code)
        references.append([reference_code])  # Each reference should be wrapped in a list

        # Print each result
        print(f"Prompt: {prompt}")
        print(f"Generated Code: {generated_code}")
        print(f"Reference Code: {reference_code}")
        print("-" * 50)

    # Calculate BLEU score across the sample
    bleu_score = corpus_bleu(predictions, references).score
    print(f"Average BLEU Score: {bleu_score}")

    return bleu_score

# Run BLEU evaluation on a sample of the test dataset
avg_bleu_score = evaluate_bleu_sample(subset_test_dataset, sample_size=6)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header of a personal portfolio website with a modern, clean aesthetic. Utilize HTML for logo, navigation bar, and concise intro sections, each with distinct calls-to-action. Employ CSS to emphasize typography, color contrasts, and hover interactions, focusing on readability and intuitive layout for seamless navigation while aligning with UI/UX best practices.


Generated Code: Create a header of a personal portfolio website with a modern, clean aesthetic. Utilize HTML for logo, navigation bar, and concise intro sections, each with distinct calls-to-action. Employ CSS to emphasize typography, color contrasts, and hover interactions, focusing on readability and intuitive layout for seamless navigation while aligning with UI/UX best practices.

**Header Section:**
 
  -	*	Logo: "Your Name" 
  -	*	Responsive navigation
  -	*	Responsive intro sections

**Header Styles:**
 
  -	*	`: 1.5rem;`		*   font-size 
  -	*	`color: #333;`		*   text color 
  -	*	`text-align: center;`		*

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header of a Photography Portfolio Website, focused on showcasing minimalistic and elegant visuals. Integrate HTML components for smooth navigation, header images, and call to action buttons. Use CSS for image overlays, typography accents, spacing, and hover effects. Prioritize visual storytelling, contrast, clean aesthetics, and alignment to create an immersive experience that follows key UI/UX principles, enhancing user engagement and retention.


Generated Code: Create a header of a Photography Portfolio Website, focused on showcasing minimalistic and elegant visuals. Integrate HTML components for smooth navigation, header images, and call to action buttons. Use CSS for image overlays, typography accents, spacing, and hover effects. Prioritize visual storytelling, contrast, clean aesthetics, and alignment to create an immersive experience that follows key UI/UX principles, enhancing user engagement and retention.

**Header Section:**
```html
header {
  background: ur

...

Prompt: Create a header for a Portfolio website under the Creative Professional subcategory with a Bold and Modern design. Integrate hero text, a captivating call to action button, and clear typography hierarchy. Utilize flexbox for layout, maintain a consistent color palette, employ whitespace for clarity, and ensure responsive design with attention to alignment, padding, and interactive hover states to elevate user experience.


Generated Code: Create a header for a Portfolio website under the Creative Professional subcategory with a Bold and Modern design. Integrate hero text, a captivating call to action button, and clear typography hierarchy. Utilize flexbox for layout, maintain a consistent color palette, employ whitespace for clarity, and ensure responsive design with attention to alignment, padding, and interactive hover states to elevate user experience.

**Portfolio** 
    <div class="portfolio">
        <div class="container">
            <div class="row">
                <d

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header for a portfolio website in the "Freelance Designer" category, featuring a sleek, modern layout with a logo, navigation menu, and a call-to-action button using HTML and CSS; ensure a responsive flexbox design, consistent typography ('Poppins' and 'Work Sans'), color contrast, padding, and spacing for an optimal UI/UX.


Generated Code: Create a header for a portfolio website in the "Freelance Designer" category, featuring a sleek, modern layout with a logo, navigation menu, and a call-to-action button using HTML and CSS; ensure a responsive flexbox design, consistent typography ('Poppins' and 'Work Sans'), color contrast, padding, and spacing for an optimal UI/UX.

**Header Section:**
 
```html
  <header class="header">
    <div class="container flex flex-col items-center p-4 gap-6">
      <a class="text-3xl font-Poppins text-white-100" href="#">Freelance Designer</a>
      <nav class="flex justify-center gap-4">
        <a href="#" class="text-white-100 hover:te

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header of a Personal Portfolio Website with a modern and minimalistic look. Utilize HTML and CSS to feature a hero section, navigation menu, profile image, service highlights, and social media integration. Use call to action buttons like Download CV and Contact Me to enhance UX. Ensure all components align with UI/UX principles for clarity and engagement.


Generated Code: Create a header of a Personal Portfolio Website with a modern and minimalistic look. Utilize HTML and CSS to feature a hero section, navigation menu, profile image, service highlights, and social media integration. Use call to action buttons like Download CV and Contact Me to enhance UX. Ensure all components align with UI/UX principles for clarity and engagement.

**Header 1.0.0
  1. 	0.0.0.1
  1. 	0.0.0.2
  1. 	0.0.0.3
  1. 	0.0.0.4
  1. 	0.0.0.5
  1. 	0.0.0.6
  1. 	0.0.0.7
  1. 	0.0.0.8
  1. 	0.0.0.9
  1. 	0.0.0.10
  1. 	0.0.0.11
  1. 	0.0.0.12
  1. 	0.0.0.13
  1. 	0.0.0.14
  1. 	0.0.0.15
  1. 	0.

###Pass@1

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from bs4 import BeautifulSoup
import re
import json
import torch

# Load the tokenizer and model
tokenizer_path = "StarryStudioAI/MT_fine_tuned_model"
model_path = "StarryStudioAI/M_fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to('cpu')  # Use 'cuda' if GPU is available

# Load the subset_test_dataset JSON file
with open('StarryStudioAI/starcoder_training_data_200.json', 'r') as f:
    subset_test_dataset = json.load(f)

# Function to extract relevant criteria from prompt
def extract_relevant_criteria(prompt):
    relevant_criteria = {
        "has_header_container": "header" in prompt.lower(),
        "has_basic_html_structure": "html structure" in prompt.lower(),
        "has_navigation_menu": "navigation" in prompt.lower() or "menu" in prompt.lower(),
        "has_responsive_design": "responsive" in prompt.lower(),
        "includes_css_styling": "css" in prompt.lower() or "style" in prompt.lower(),
        "has_call_to_action": "call-to-action" in prompt.lower() or "button" in prompt.lower(),
        "uses_semantic_tags": "semantic" in prompt.lower() or "structure" in prompt.lower(),
        "no_syntax_errors": True  # Generalized for all prompts, assuming BeautifulSoup parse without errors
    }
    return {k: v for k, v in relevant_criteria.items() if v}

# Success criteria definition for Pass@1 evaluation
def evaluate_pass_at_1(generated_code, relevant_criteria):
    soup = BeautifulSoup(generated_code, 'html.parser')

    # Generalized Criteria checks based on relevant criteria
    results = {}
    if relevant_criteria.get("has_header_container"):
        results["has_header_container"] = bool(soup.find('header') or soup.find('div', {'class': re.compile('header', re.IGNORECASE)}))
    if relevant_criteria.get("has_basic_html_structure"):
        results["has_basic_html_structure"] = all([
            bool(soup.find('html')),
            bool(soup.find('head')),
            bool(soup.find('body'))
        ])
    if relevant_criteria.get("has_navigation_menu"):
        results["has_navigation_menu"] = bool(soup.find('nav') or soup.find('ul') or soup.find_all('a', href=True))
    if relevant_criteria.get("has_responsive_design"):
        results["has_responsive_design"] = '@media' in generated_code or bool(soup.find('meta', {'name': 'viewport'}))
    if relevant_criteria.get("includes_css_styling"):
        results["includes_css_styling"] = '<style>' in generated_code or 'stylesheet' in generated_code or 'style=' in generated_code
    if relevant_criteria.get("has_call_to_action"):
        results["has_call_to_action"] = bool(soup.find('a', {'class': re.compile('button|cta', re.IGNORECASE)}) or soup.find('button'))
    if relevant_criteria.get("uses_semantic_tags"):
        results["uses_semantic_tags"] = any(tag.name in ['header', 'nav', 'section'] for tag in soup.find_all())
    if relevant_criteria.get("no_syntax_errors"):
        results["no_syntax_errors"] = True  # BeautifulSoup parsing indicates no syntax errors

    # Check if all relevant criteria are met for Pass@1
    pass_at_1 = all(results.values())
    return pass_at_1, results

def evaluate_pass_at_1_subset(subset_test_dataset, num_samples=6):
    sample_dataset = subset_test_dataset[:num_samples]  # Limit to num_samples

    pass_count = 0
    detailed_logs = []

    for example in sample_dataset:
        prompt = example['prompt']
        reference_code = example['completion']

        # Extract relevant criteria from the prompt
        relevant_criteria = extract_relevant_criteria(prompt)

        # Tokenize and generate output
        inputs = tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=1024)
        generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

        # Evaluate Pass@1 criteria and log results
        pass_at_1, criteria_results = evaluate_pass_at_1(generated_code, relevant_criteria)

        # Count the passes
        if pass_at_1:
            pass_count += 1

        # Log details for this example
        detailed_logs.append({
            "prompt": prompt,
            "generated_code": generated_code,
            "reference_code": reference_code,
            "pass_at_1": "Passed" if pass_at_1 else "Failed",
            "criteria_results": criteria_results  # Detailed criteria outcomes
        })

        # Print each result
        print(f"Prompt: {prompt}")
        print(f"Generated Code:\n{generated_code}")
        print(f"Reference Code:\n{reference_code}")
        print("Pass@1:", "Passed" if pass_at_1 else "Failed")
        print("Relevant Criteria Evaluated:", relevant_criteria)
        print("Detailed Criteria Results:")
        for criterion, passed in criteria_results.items():
            status = "Pass" if passed else "Fail"
            print(f"  {criterion}: {status}")
        print("-" * 50)

    # Calculate Pass@1 rate
    pass_at_1_rate = (pass_count / len(sample_dataset)) * 100 if sample_dataset else 0
    print(f"Pass@1 Rate for {num_samples} samples: {pass_at_1_rate:.2f}%")

    # Log the overall summary
    overall_log = {
        "pass_at_1_rate": pass_at_1_rate,
        "details": detailed_logs
    }

    return pass_at_1_rate, overall_log

# Test on only one example
print("Evaluating Pass@1 on one sample...")
pass_at_1_rate_single, log_single = evaluate_pass_at_1_subset(subset_test_dataset, num_samples=6)

# Uncomment to evaluate on all examples in the subset after testing
'''
print("\nEvaluating Pass@1 on all samples...")
pass_at_1_rate_all, log_all = evaluate_pass_at_1_subset(subset_test_dataset, num_samples=len(subset_test_dataset))
'''


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Evaluating Pass@1 on one sample...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header of a personal portfolio website with a modern, clean aesthetic. Utilize HTML for logo, navigation bar, and concise intro sections, each with distinct calls-to-action. Employ CSS to emphasize typography, color contrasts, and hover interactions, focusing on readability and intuitive layout for seamless navigation while aligning with UI/UX best practices.


Generated Code:
Create a header of a personal portfolio website with a modern, clean aesthetic. Utilize HTML for logo, navigation bar, and concise intro sections, each with distinct calls-to-action. Employ CSS to emphasize typography, color contrasts, and hover interactions, focusing on readability and intuitive layout for seamless navigation while aligning with UI/UX best practices.

```html
 .topnav {
  text-align: center;
  position: relative;
  z-index: 1;
  width: 100%;
  display: flex;
  justify-content: space-around;
  flex-direction: column;
  color: #000;
  font-family: Arial, sans-serif;
  font-size: 1

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header for a portfolio website in the "Freelance Designer" category, featuring a sleek, modern layout with a logo, navigation menu, and a call-to-action button using HTML and CSS; ensure a responsive flexbox design, consistent typography ('Poppins' and 'Work Sans'), color contrast, padding, and spacing for an optimal UI/UX.


Generated Code:
Create a header for a portfolio website in the "Freelance Designer" category, featuring a sleek, modern layout with a logo, navigation menu, and a call-to-action button using HTML and CSS; ensure a responsive flexbox design, consistent typography ('Poppins' and 'Work Sans'), color contrast, padding, and spacing for an optimal UI/UX.

```css
/* Header Styles */
header {
  display: flex;
  justify-content: center;
  align-items: center;
  min-height: 100vh;
  height: 100%;
  padding: 40px 0;
  background: #f7f7f7;
  box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
  color: #333;
}

/* Logo Styles */
header img {
  width: 100px;
  height: 100

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt: Create a header of a Personal Portfolio Website with a modern and minimalistic look. Utilize HTML and CSS to feature a hero section, navigation menu, profile image, service highlights, and social media integration. Use call to action buttons like Download CV and Contact Me to enhance UX. Ensure all components align with UI/UX principles for clarity and engagement.


Generated Code:
Create a header of a Personal Portfolio Website with a modern and minimalistic look. Utilize HTML and CSS to feature a hero section, navigation menu, profile image, service highlights, and social media integration. Use call to action buttons like Download CV and Contact Me to enhance UX. Ensure all components align with UI/UX principles for clarity and engagement.

**Header Structure:**

*   **Hero Section:**
    *   Background image or video
    *     Text 
    *       -   800px
    *     -   600px
    *     -   400px
    *   100px
    *   100px
*   **Navigation Menu:**
    *   100px
*   **Profile I

'\nprint("\nEvaluating Pass@1 on all samples...")\npass_at_1_rate_all, log_all = evaluate_pass_at_1_subset(subset_test_dataset, num_samples=len(subset_test_dataset))\n'